In [2]:
import os
import json
from dotenv import load_dotenv
from langchain_chroma import Chroma
import google.generativeai as genai
from langchain.llms import GooglePalm
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.document_loaders import JSONLoader
from langchain_google_genai import GoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

import warnings
warnings.filterwarnings("ignore")

# Load environment variables and configure Google Generative AI
load_dotenv()
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Initialize the Google Generative AI model
# llm = GoogleGenerativeAI(model="models/text-bison-001", temperature=0.1)
llm = GooglePalm(model_name="models/text-bison-001", temperature=0.2)

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

import warnings
warnings.filterwarnings("ignore")

# Load the embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Now try to load the vector store
loaded_vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
print(f"Number of documents in the loaded vector store: {loaded_vectorstore._collection.count()}")

Number of documents in the loaded vector store: 47


In [4]:
# from mdpdf import markdown_to_pdf

# Updated prompt template to specify Markdown output
prompt_template = """You are an AI assistant tasked with writing a comprehensive document in Markdown format based on a provided table of contents. 
Use the following pieces of context to write detailed sections for the document.
If you don't have enough information, state that more research is needed on that topic.

Context: {context}

Section to write:
{question}

AI Assistant: Write a detailed section for the given header in Markdown format. 
Provide comprehensive and informative content directly related to the section title.
Do not create additional headers or a table of contents.
Ensure the content is well-structured, relevant to the topic, and flows logically."""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Update the qa chain with the new prompt
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=loaded_vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

def notegen(query):
    result = qa.invoke({"query": query})
    return result

# answer2 = notegen(query = "##1. Fundamentals of LLM")
# answer3 = notegen(query = "1.1 Definition and Key Concepts")
# answer1 = notegen(query = """#Understanding Large Language Model Architectures""")

# Convert Markdown content to PDF
# pdf_filename = "output_document.pdf"
# markdown_to_pdf(markdown_content, pdf_filename)

In [5]:
answer1 = notegen(query = """#Understanding Large Language Model Architectures""")

E0729 07:41:27.147885000 4720809472 client_channel.cc:625]             chand=0x7fb079ccccf0: Illegal keepalive throttling value 
E0729 07:41:27.147921000 4720809472 client_channel.cc:625]             chand=0x7fb079ccccf0: Illegal keepalive throttling value 
E0729 07:41:27.147927000 4720809472 client_channel.cc:625]             chand=0x7fb079ccccf0: Illegal keepalive throttling value 
E0729 07:41:27.147931000 4720809472 client_channel.cc:625]             chand=0x7fb079ccccf0: Illegal keepalive throttling value 
E0729 07:41:27.147936000 4720809472 client_channel.cc:625]             chand=0x7fb079ccccf0: Illegal keepalive throttling value 
E0729 07:41:27.147940000 4720809472 client_channel.cc:625]             chand=0x7fb079ccccf0: Illegal keepalive throttling value 
E0729 07:41:27.147943000 4720809472 client_channel.cc:625]             chand=0x7fb079ccccf0: Illegal keepalive throttling value 
E0729 07:41:27.147947000 4720809472 client_channel.cc:625]             chand=0x7fb079ccccf0: Ille

In [6]:
answer3 = notegen(query = "1.1 Definition and Key Concepts")

In [7]:
answer2 = notegen(query = """#Fundamentals of LLMs""")

IndexError: list index out of range

: 

In [ ]:
answer1

{'query': '#Understanding Large Language Model Architectures',
 'result': '## Understanding Large Language Model Architectures\n\nLarge language models (LLMs) are deep neural networks that have been trained on massive datasets of text, code, or other structured data. They are able to perform a wide variety of natural language processing (NLP) tasks, such as text generation, summarization, translation, and question answering.\n\nLLMs are typically built on top of the transformer architecture, which was first introduced in the paper "Attention Is All You Need". Transformers are a type of neural network that is specifically designed for processing sequential data, such as text. They work by attending to different parts of the input sequence and then generating an output based on these attended parts.\n\nThe transformer architecture has been shown to be very effective for NLP tasks, and it is now the basis of many of the most successful LLMs. However, transformers are also very computation

In [ ]:
answer2

NameError: name 'answer2' is not defined

In [ ]:
answer3

{'query': '1.1 Definition and Key Concepts',
 'result': '## 1.1 Definition and Key Concepts\n\nA large language model (LLM) is a type of neural network that has been trained on a massive dataset of text, typically in the order of billions of words. This allows LLMs to learn the relationships between words and phrases, and to generate text that is both coherent and fluent. LLMs have been used for a variety of tasks, including natural language processing, machine translation, and code generation.\n\n### Key Concepts\n\n* **Context:** The context is the text that is provided to the LLM before it generates output. The context can be used to provide the LLM with information about the topic of the output, or to give it a starting point for its generation.\n* **Prompt:** The prompt is the text that is provided to the LLM after the context. The prompt can be used to provide the LLM with additional instructions or constraints on its output.\n* **Output:** The output is the text that is generate

In [ ]:
# Create a custom prompt template
prompt_template = """You are an AI assistant tasked with writing a comprehensive document based on a provided table of contents. 
Your goal is to generate detailed content for each section and subsection of the document.

Table of Contents:
{toc}

AI Assistant: Write a detailed document covering all the sections and subsections provided in the table of contents above. 
For each main section (##) and subsection (###), provide comprehensive and informative content. 
Ensure the content is well-structured, relevant to the topic, and flows logically from one section to the next. 
Do not include the section or subsection headers in your response, as these will be added separately.

If you don't have enough information on a particular topic, state that more research is needed for that specific section or subsection.

Begin writing the document now, starting with the content for the first main section."""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Create a retrieval QA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=loaded_vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# query = "Introduction: What is the purpose of this document?"
# result = qa.invoke({"query": query})

def notegen(query):
    result = qa.invoke({"query": query})
    return result

In [ ]:
answer = notegen(query="#Understanding Large Language Model Architectures, ##1. Fundamentals of LLMs, ###1.1 Definition and Key Concepts, ###1.2 Historical Development of LLMs, ##2. Core Architectural Components, ###2.1 Transformer Architecture, ###2.2 Attention Mechanism, ###2.3 Self-Attention Mechanism, ##3. Training and Fine-Tuning LLMs, ###3.1 Data Preprocessing, ###3.2 Training Process, ###3.3 Fine-Tuning Process, ##4. Applications of LLMs, ###4.1 Natural Language Processing, ###4.2 Text Generation, ###4.3 Machine Translation, ##5. Challenges and Limitations, ###5.1 Ethical Concerns, ###5.2 Bias and Fairness, ###5.3 Computational Resources, ##6. Future Directions, ###6.1 Advancements in LLMs, ###6.2 Research and Development, ###6.3 Potential Applications")

In [ ]:
answer

In [ ]:
prompt_template = """You are an AI assistant tasked with writing a comprehensive document based on a provided table of contents. 
Use the following pieces of context to write detailed sections for the document.
If you don't have enough information, state that more research is needed on that topic.

Context: {context}

Table of Contents:
{question}

AI Assistant: Write a detailed document covering all the sections and subsections provided in the table of contents above. 
For each main section (##) and subsection (###), provide comprehensive and informative content. 
Ensure the content is well-structured, relevant to the topic, and flows logically from one section to the next. 
Do not include the section or subsection headers in your response, as these will be added separately."""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Update the qa chain with the new prompt
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=loaded_vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

def notegen(query):
    result = qa.invoke({"query": query})
    return result

# Now you can use notegen as before
answer = notegen(query="#Understanding Large Language Model Architectures, ##1. Fundamentals of LLMs, ###1.1 Definition and Key Concepts, ###1.2 Historical Development of LLMs, ##2. Core Architectural Components, ###2.1 Transformer Architecture, ###2.2 Attention Mechanism, ###2.3 Self-Attention Mechanism, ##3. Training and Fine-Tuning LLMs, ###3.1 Data Preprocessing, ###3.2 Training Process, ###3.3 Fine-Tuning Process, ##4. Applications of LLMs, ###4.1 Natural Language Processing, ###4.2 Text Generation, ###4.3 Machine Translation, ##5. Challenges and Limitations, ###5.1 Ethical Concerns, ###5.2 Bias and Fairness, ###5.3 Computational Resources, ##6. Future Directions, ###6.1 Advancements in LLMs, ###6.2 Research and Development, ###6.3 Potential Applications")

In [ ]:
answer